### Workbook for exploring catchment survey data and replicating methods from 2021 paper
Week of June 9, 2025
<br>
Author: Adele Birkenes

Objectives of 2021 study (pg. 5):
- Understand the geographic area served by a completed bridge and validate the current needs assessment approach by comparing the geographic area served to the estimate of individuals directly served determined as part of the needs assessment
- Understand the specific ways in which completed trailbridges serve as connections to key destinations

The GIS methods in the paper are as follows:
<br>
5.1 Calculate summary statistics on number of individuals surveyed, demographics (age range and gender), and modes of transportation

5.2 Calculate total catchment area of each site in sq km, and count number of unique villages served by the site. Compare number of unique villages to "qualified home villages" from the needs assessment.
- Calculate the proportion of respondents reporting a given home village or destination
- Calculate mean Euclidean distance from bridge to centroid of farthest home village with one or more surveys
- Determine which of the home villages from the catchment surveys are considered "qualified" based on whether they meet hte criteria for individuals directly served, based on reported purpose of travel

Sample outputs:

| Bridge Site | Catchment area, all home and destination villages | Catchment area (sq km) | Catchment area, qualified home villages | Qualified home villages (needs assessment) |
|-------------|---------------------------------------------------|------------------------|------------------------------------------|---------------------------------------------|
| Gahororo    | 27                                                | 44.6                   | 18                                       | 4                                           |

<br>
<img src="graphics/Figure4.png" alt="Figure 4" width="400"/>
<br>
<img src="graphics/Figure5.png" alt="Figure 5" width="400"/>

5.3 Calculate summary statistics on purpose of travel, disaggregated by gender

5.4 Calculate travel time for a one-way trip between home village and destination, disaggregated by purpose of travel and gender
- Exclude multi-day trips